# Tarefa 1: DATA APP para Classificador Multiclasse MNIST

Este notebook segue os passos da avaliação:
a) Avaliar 7 modelos com `cross_val_predict`.
b) Ranquear o top 3 por acurácia de CV.
c) Aplicar os 7 modelos no conjunto de teste inicial.
d) Ranquear o top 3 pela menor diferença (CV vs. Teste) e analisar.
e) Realizar `RandomizedSearchCV` no **Top 1** (conforme sugestão do professor para acelerar).
f) Aplicar o melhor modelo de 'e' em um *novo* conjunto de teste.
g) Exportar o melhor modelo final e o scaler.

### Célula 1: Importação das Bibliotecas

Aqui, importamos todas as ferramentas necessárias. Isso inclui `numpy` e `pandas` para manipulação de dados, `matplotlib` para gráficos, `sklearn` para carregar dados, pré-processar (StandardScaler), dividir dados (train_test_split), avaliar (cross_val_predict, accuracy_score) e os 7 modelos de classificação solicitados.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_predict, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Modelos para a etapa 1a
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')
np.random.seed(42) # Define uma semente aleatória para reprodutibilidade

### Célula 2: Carga e Divisão Inicial dos Dados

Carregamos o dataset MNIST (70.000 imagens 28x28). Seguindo a convenção padrão do MNIST, dividimos os dados:
* `X_train_orig` / `y_train_orig`: Os primeiros 60.000 exemplos para treino e validação cruzada.
* `X_test1` / `y_test1`: Os últimos 10.000 exemplos como nosso primeiro conjunto de teste (para a etapa 1c).

In [ ]:
# Carregar dados
mnist = fetch_openml('mnist_784', version=1, as_frame=True, parser='auto')
X = mnist.data
y = mnist.target.astype(np.uint8)

# Divisão original (como nos notebooks de exemplo) - Teste 1
X_train_orig, X_test1, y_train_orig, y_test1 = X[:60000], X[60000:], y[:60000], y[60000:]

### Célula 3: Padronização dos Dados (Scaler)

Muitos modelos (como SVM, MLP, KNN, Regressão Logística) são sensíveis à escala dos dados (pixels de 0 a 255). Usamos `StandardScaler` para padronizar os dados (média 0, desvio padrão 1).

**Importante:** Ajustamos (`fit_transform`) o scaler *apenas* nos dados de treino (`X_train_orig`) e depois usamos esse mesmo scaler (com a média e desvio do treino) para transformar (`transform`) o conjunto de teste (`X_test1`).

In [ ]:
# Escalar os dados (essencial para muitos desses modelos)
scaler_orig = StandardScaler()
X_train_orig_scaled = scaler_orig.fit_transform(X_train_orig)
X_test1_scaled = scaler_orig.transform(X_test1)

print(f"Formato Treino Original: {X_train_orig_scaled.shape}")
print(f"Formato Teste 1: {X_test1_scaled.shape}")

### Célula 4: Etapas 1a & 1b - Avaliação com Validação Cruzada (CV)

**Etapa 1a:** Definimos os 7 modelos solicitados, todos com hiperparâmetros *default* (padrão) e `random_state=42` para garantir que os resultados sejam os mesmos em diferentes execuções.

**Etapa 1b:** Iteramos sobre cada modelo:
1.  Usamos `cross_val_predict(cv=3)` para fazer uma validação cruzada de 3 folds. Isso treina o modelo em 2/3 dos dados e testa no 1/3 restante, repetindo 3 vezes até que todos os dados de treino tenham sido usados como teste uma vez.
2.  Calculamos a `accuracy_score` (acurácia global) comparando as predições da CV (`y_train_pred`) com os rótulos reais do treino (`y_train_orig`).
3.  Armazenamos os resultados, ordenamos do melhor para o pior e identificamos o **Top 3**.

In [ ]:
# Definir os 7 modelos com hiperparâmetros default e random_state para reprodutibilidade
models = {
    'GaussianNB': GaussianNB(),
    'MLPClassifier': MLPClassifier(random_state=42, max_iter=300), # Aumentar max_iter para evitar warning
    'SVC': SVC(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1),
    'SGDClassifier': SGDClassifier(random_state=42, n_jobs=-1),
    'KNeighborsClassifier': KNeighborsClassifier(n_jobs=-1),
    'XGBClassifier': XGBClassifier(random_state=42, n_jobs=-1, eval_metric='mlogloss', use_label_encoder=False)
}

results = []

print("Iniciando Etapa 1a: Avaliação com cross_val_predict...")

for name, model in models.items():
    print(f"Avaliando {name}...")
    # NOTA: O dataset completo (60k) é usado. SVC e KNN podem levar vários minutos.
    y_train_pred = cross_val_predict(model, X_train_orig_scaled, y_train_orig, cv=3, n_jobs=-1)
    cv_accuracy = accuracy_score(y_train_orig, y_train_pred)
    results.append({'Modelo': name, 'Acurácia CV': cv_accuracy})
    print(f"{name} - Acurácia CV: {cv_accuracy:.4f}")

# Criar DataFrame e ranquear (Etapa 1b)
results_df = pd.DataFrame(results).sort_values(by='Acurácia CV', ascending=False).reset_index(drop=True)

print("\n--- Etapa 1b: Ranking por Acurácia CV ---")
print(results_df)

top_3_cv_models = results_df['Modelo'].head(3).tolist()
print(f"\nTop 3 Modelos (CV): {top_3_cv_models}")

### Célula 5: Etapas 1c & 1d - Avaliação no Teste 1 e Análise de Diferença

**Etapa 1c:** Treinamos (fit) cada um dos 7 modelos no conjunto de treino *completo* (`X_train_orig_scaled`, 60k amostras) e aplicamos (predict) no conjunto de teste 1 (`X_test1_scaled`, 10k amostras), que o modelo nunca viu.

**Etapa 1d:** Calculamos a diferença absoluta (`Abs(Diferença)`) entre a Acurácia CV (Etapa 1a) e a Acurácia Teste (Etapa 1c). Ranqueamos os modelos pela menor diferença. Isso nos ajuda a ver quais modelos generalizam melhor (ou seja, têm menos overfitting).

In [ ]:
print("\nIniciando Etapa 1c: Aplicação no Conjunto de Teste 1...")
test_accuracies = []

for name, model in models.items():
    print(f"Treinando e Testando {name}...")
    model.fit(X_train_orig_scaled, y_train_orig)
    y_test1_pred = model.predict(X_test1_scaled)
    test_accuracy = accuracy_score(y_test1, y_test1_pred)
    test_accuracies.append(test_accuracy)
    print(f"{name} - Acurácia Teste 1: {test_accuracy:.4f}")

results_df['Acurácia Teste'] = test_accuracies
results_df['Diferença (CV - Teste)'] = results_df['Acurácia CV'] - results_df['Acurácia Teste']
results_df['Abs(Diferença)'] = np.abs(results_df['Diferença (CV - Teste)'])

# Ranquear por menor diferença (Etapa 1d)
results_df_diff_rank = results_df.sort_values(by='Abs(Diferença)', ascending=True)

print("\n--- Etapa 1d: Ranking por Menor Diferença (CV vs. Teste) ---")
print(results_df_diff_rank[['Modelo', 'Acurácia CV', 'Acurácia Teste', 'Abs(Diferença)']])

top_3_diff_models = results_df_diff_rank['Modelo'].head(3).tolist()
print(f"\nTop 3 Modelos (Menor Diferença): {top_3_diff_models}")

### Célula 6: 1d (Análise)

**Análise:** 
Com base nos resultados (que podem variar dependendo da execução exata e das bibliotecas):

* **Top 3 (Acurácia CV):** [A ser preenchido pela execução, ex: 'SVC', 'MLPClassifier', 'KNeighborsClassifier']
* **Top 3 (Menor Diferença):** [A ser preenchido, ex: 'SVC', 'LogisticRegression', 'XGBClassifier']

**Os modelos coincidem?**
(A ser preenchido) Frequentemente, o SVC aparece em ambas as listas, mostrando alta performance *e* boa generalização. Modelos como KNN, embora tenham alta acurácia na CV (por "memorizar" o treino), podem ter uma queda maior no teste. Modelos mais simples (como `GaussianNB` ou `SGDClassifier` default) podem ter acurácia baixa, mas também baixa diferença (underfitting), enquanto modelos complexos (como `MLP` ou `XGB` default) podem ter alta acurácia no CV, mas também alta diferença (overfitting).

### Célula 7: Etapa 1e & 1f (Preparação) - Novo Split de Dados

Conforme a instrução 'f', precisamos de um *novo* conjunto de teste. Usamos `train_test_split` em todo o dataset (X, y) com `random_state=101` (diferente da divisão original) para criar `X_train_new` e `X_test_new`.

In [ ]:
# 1. Novo Split (usando train_test_split para garantir uma semente diferente)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size=10000, random_state=101, stratify=y)


### Célula 8: Etapa 1e & 1f (Preparação) - Novo Scaler

Criamos um *novo* scaler (`scaler_new`). Ajustamos (`fit_transform`) ele *apenas* aos novos dados de treino (`X_train_new_scaled`) e o usamos para transformar (`transform`) o novo teste (`X_test_new_scaled`).

In [ ]:
# 2. Escalar novos dados
scaler_new = StandardScaler()
X_train_new_scaled = scaler_new.fit_transform(X_train_new)
X_test_new_scaled = scaler_new.transform(X_test_new)

print(f"Formato Novo Treino: {X_train_new_scaled.shape}")
print(f"Formato Novo Teste: {X_test_new_scaled.shape}")

### Célula 9: Etapa 1e - Definição dos Hiperparâmetros

Definimos os hiperparâmetros e as faixas de valores para os 3 melhores modelos da Etapa 1b (`top_3_cv_models`). Usamos distribuições (como `randint`, `loguniform`) para o `RandomizedSearchCV`, que testa combinações aleatórias, sendo mais eficiente que o `GridSearchCV`.

In [ ]:
# 3. Definir Modelos e Hiperparâmetros para RandomizedSearch (Top 3 da Etapa 1b)

from scipy.stats import randint, uniform, loguniform

# Dicionário completo de grids de parâmetros
param_grids = {
    'MLPClassifier': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'alpha': loguniform(1e-4, 1e-1),
        'learning_rate_init': loguniform(1e-4, 1e-2)
    },
    'SVC': {
        'C': loguniform(1e-1, 1e2),
        'gamma': loguniform(1e-4, 1e-1),
        'kernel': ['rbf', 'poly']
    },
    'KNeighborsClassifier': {
        'n_neighbors': randint(3, 10),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'LogisticRegression': {
        'C': loguniform(1e-2, 1e2),
        'solver': ['saga'],
        'penalty': ['l1', 'l2']
    },
    'SGDClassifier': {
        'loss': ['hinge', 'log_loss'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': loguniform(1e-5, 1e-1)
    },
    'XGBClassifier': {
        'n_estimators': randint(100, 500),
        'learning_rate': loguniform(0.01, 0.3),
        'max_depth': randint(3, 10)
    },
    'GaussianNB': {
        'var_smoothing': loguniform(1e-10, 1e-8)
    }
}

# Constantes para a busca (Conforme instrução 'e')
N_COMBINACOES = 10 # N=10 combinações
CV_FOLDS = 3       # Mesmo número de folds
RANDOM_SEED = 42   # Mesma semente

best_estimators = []

print(f"\nIniciando Etapa 1e: RandomizedSearchCV para o Top 1 (Otimização de tempo)...")
print(f"Melhor modelo da Etapa 1b: {top_3_cv_models[0]}")

### Célula 10: Etapa 1e (Execução) - RandomizedSearchCV (Otimizado)

Conforme a sugestão do professor para acelerar o processo, executamos o `RandomizedSearchCV` **apenas no melhor modelo (Top 1)** da Etapa 1b.

1.  Configuramos o `RandomizedSearchCV` com N=10, CV=3, Semente=42.
2.  Executamos o `fit` no *novo* conjunto de treino (`X_train_new_scaled`).
3.  Salvamos o `best_estimator_` para a próxima etapa.

In [ ]:
# --- INÍCIO DA MODIFICAÇÃO (Baseada na Sugestão do Professor) ---
# Em vez de iterar nos 3, pegamos apenas o melhor (índice 0)
best_model_name = top_3_cv_models[0]

if best_model_name not in models or best_model_name not in param_grids:
    print(f"Modelo {best_model_name} não encontrado. Parando.")
else:
    print(f"Iniciando busca para {best_model_name}...")
    base_model = models[best_model_name]
    params = param_grids[best_model_name]
    
    random_search = RandomizedSearchCV(
        base_model,
        param_distributions=params,
        n_iter=N_COMBINACOES,
        cv=CV_FOLDS,
        random_state=RANDOM_SEED,
        n_jobs=-1,
        scoring='accuracy'
    )
    
    # 4. Executar a busca no *novo* treino
    random_search.fit(X_train_new_scaled, y_train_new)
    
    print(f"Melhores parâmetros para {best_model_name}: {random_search.best_params_}")
    print(f"Melhor acurácia CV ({best_model_name}): {random_search.best_score_:.4f}")
    
    # Salva o único estimador otimizado
    best_estimators = [random_search.best_estimator_]

print("\nBusca de Hiperparâmetros Concluída.")
# --- FIM DA MODIFICAÇÃO ---

### Célula 11: Etapa 1f - Avaliação do Modelo Otimizado

Agora, pegamos o único modelo otimizado (da Etapa 1e) e o aplicamos no `X_test_new_scaled` (o conjunto de teste que foi separado na Etapa 1e e não foi usado na otimização). 

Isso nos dá a performance final do nosso melhor modelo em dados completamente novos.

In [ ]:
print("\nIniciando Etapa 1f: Avaliação no Novo Conjunto de Teste...")
final_results = []

# Esta lista agora deve conter apenas 1 estimador
if not best_estimators:
    print("Nenhum modelo foi otimizado. Parando.")
else:
    model = best_estimators[0]
    model_name = model.__class__.__name__
    y_test_new_pred = model.predict(X_test_new_scaled)
    test_accuracy = accuracy_score(y_test_new, y_test_new_pred)
    
    print(f"\n--- {model_name} (Otimizado) ---")
    print(f"Acurácia no Novo Teste: {test_accuracy:.4f}")
    print(classification_report(y_test_new, y_test_new_pred))
    
    final_results.append({'Modelo': model_name, 'Acurácia Novo Teste': test_accuracy, 'Estimador': model})

# Determinar o melhor modelo final
final_df = pd.DataFrame(final_results).sort_values(by='Acurácia Novo Teste', ascending=False)
best_overall_model_info = final_df.iloc[0]
best_overall_model = best_overall_model_info['Estimador']

print("\n--- Melhor Modelo Geral (Pós-Otimização) ---")
print(best_overall_model_info[['Modelo', 'Acurácia Novo Teste']])

### Célula 12: Etapa 1g - Exportar o Melhor Modelo e Scaler

Finalmente, pegamos o melhor modelo da Etapa 1f (`best_overall_model`) e o `scaler_new` (o scaler que foi ajustado aos dados de treino desse modelo) e os salvamos em arquivos `.joblib`. Esses arquivos podem ser carregados pela aplicação Streamlit.

In [ ]:
model_filename = 'best_mnist_model.joblib'
scaler_filename = 'mnist_scaler.joblib'

if best_estimators: # Só salva se o modelo foi treinado
    joblib.dump(best_overall_model, model_filename)
    joblib.dump(scaler_new, scaler_filename) # Salvar o scaler que foi usado no treino deste modelo

    print(f"\nEtapa 1g Concluída.")
    print(f"Melhor modelo salvo em: {model_filename}")
    print(f"Scaler correspondente salvo em: {scaler_filename}")
else:
    print("\nEtapa 1g Falhou: Nenhum modelo foi treinado/otimizado.")